# Test Checkpoint-250 Model

This notebook tests your checkpoint-250 model from Colab.

**Setup:**
1. Download `checkpoint-250` folder from Colab (`/content/n8n-workflow-generator/checkpoint-250/`)
2. Place it in `models/checkpoint-250/` in this project
3. Run all cells below to test the model

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU (will be slower)")

In [ ]:
# Install required packages if needed
!pip install -q transformers peft torch bitsandbytes accelerate

In [ ]:
# Load the base model and checkpoint
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import os

# Paths
base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
checkpoint_path = r"C:\Users\Nike\Documents\Programming\Projects\N8N\models\checkpoint-250"

# Check if checkpoint exists
if not os.path.exists(checkpoint_path):
    print(f"❌ Checkpoint not found at: {checkpoint_path}")
    print("\n📥 Please download checkpoint-250 from Colab and place it here.")
    raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")

print(f"✓ Found checkpoint at: {checkpoint_path}")
print("\nLoading base model...")

# 4-bit quantization config (same as training)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

print("Loading LoRA adapters from checkpoint-250...")
# Load LoRA adapters from checkpoint
model = PeftModel.from_pretrained(base_model, checkpoint_path)

print("✅ Model loaded successfully!")
print(f"Model size: {model.get_memory_footprint() / 1e9:.2f} GB")

In [ ]:
# Test function
def generate_workflow(prompt, max_tokens=1024, temperature=0.7):
    """Generate a workflow from a prompt"""
    full_prompt = f"""<|system|>
You are an n8n workflow generator. Convert natural language descriptions into valid n8n workflow JSON.
<|user|>
{prompt}
<|assistant|>
"""
    
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    
    print(f"🔄 Generating workflow for: '{prompt}'")
    print("⏳ This may take 30-60 seconds...\n")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        do_sample=True,
        top_p=0.95,
        repetition_penalty=1.1,
    )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    workflow = generated_text.split("<|assistant|>")[-1].strip()
    
    return workflow

print("✓ Test function ready!")

## Test 1: Simple Email Workflow

In [ ]:
workflow = generate_workflow("Create a workflow that sends an email notification")
print("="*80)
print("GENERATED WORKFLOW:")
print("="*80)
print(workflow)
print("="*80)

## Test 2: HTTP Request Workflow

In [ ]:
workflow = generate_workflow("Create a workflow that makes an HTTP request and processes the response")
print("="*80)
print("GENERATED WORKFLOW:")
print("="*80)
print(workflow)
print("="*80)

## Test 3: Slack Integration

In [ ]:
workflow = generate_workflow("Create a workflow that posts a message to Slack")
print("="*80)
print("GENERATED WORKFLOW:")
print("="*80)
print(workflow)
print("="*80)

## Test 4: Complex Multi-Step Workflow

In [ ]:
workflow = generate_workflow("Create a workflow that fetches data from an API, filters it, and saves to a database")
print("="*80)
print("GENERATED WORKFLOW:")
print("="*80)
print(workflow)
print("="*80)

## Test 5: Custom Prompt (Edit and Run)

In [ ]:
# Edit the prompt below and run to test your own ideas
custom_prompt = "Create a workflow that monitors a folder for new files"

workflow = generate_workflow(custom_prompt)
print("="*80)
print("GENERATED WORKFLOW:")
print("="*80)
print(workflow)
print("="*80)

## Validate JSON Output

In [ ]:
import json

def validate_workflow_json(workflow_text):
    """Try to parse the workflow as JSON"""
    try:
        # Extract JSON from the text (might have extra text)
        start = workflow_text.find('{')
        end = workflow_text.rfind('}') + 1
        if start == -1 or end == 0:
            print("❌ No JSON found in output")
            return False
        
        json_str = workflow_text[start:end]
        parsed = json.loads(json_str)
        
        print("✅ Valid JSON!")
        print(f"✓ Has 'nodes': {'nodes' in parsed}")
        print(f"✓ Has 'connections': {'connections' in parsed}")
        if 'nodes' in parsed:
            print(f"✓ Number of nodes: {len(parsed['nodes'])}")
        return True
    except json.JSONDecodeError as e:
        print(f"❌ Invalid JSON: {e}")
        return False

# Test the last generated workflow
print("Testing last generated workflow...")
validate_workflow_json(workflow)